In [1]:
import nltk

In [3]:
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt

In [5]:
from google.colab import files
uploaded = files.upload()

Saving sample_qns.csv to sample_qns.csv


In [6]:
data=pd.read_csv('sample_qns.csv')

In [7]:
data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
lem = WordNetLemmatizer()
ps = PorterStemmer()

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
import numpy as np
stop_words = set(stopwords.words('english'))
def remove_punc(data):
  symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in symbols:
    data = np.char.replace(data, i, ' ')
  
def remove_apostrophe(data):
  return np.char.replace(data, "'", "")



In [12]:
data['question1']=data['question1'].apply(lambda x:x.lower())
data['question2']=data['question2'].apply(lambda x:x.lower())

In [13]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,3,4,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...,0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,7,8,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?,0


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Cleaning and preprocessing text

In [34]:
#cleaning and text pre processing
for j in ['question1','question2']:
   data[j] = data[j].str.replace('[^\w\s]','')
   data[j]=data[j].str.split(' ')
   data[j]=data[j].apply(lambda x:[lem.lemmatize(y) for y in x if y not in stop_words])
   data[j]=data[j].apply(lambda x:[ps.stem(y) for y in x])


In [35]:
data['question1']=[" ".join(q) for q in data['question1'].values]
data['question2']=[" ".join(q) for q in data['question2'].values]

In [36]:
tf=TfidfVectorizer()
questions = pd.Series(pd.concat([data['question1'],data['question2']]),dtype=str)
X_tfdf = tf.fit(questions)

In [37]:
X_q1_tfidf = tf.transform(data.question1)
X_q2_tfidf = tf.transform(data.question2)

In [51]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1, feature_vec_2)

In [52]:
X = get_cosine_similarity(X_q1_tfidf,X_q2_tfidf)
y = data['is_duplicate']

In [53]:
print(X.shape)
print(y.shape)

(3966, 3966)
(3966,)


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Logistic Regression

In [56]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
pred_lr = lr.predict_proba(X_test)[:,1]
logloss_lr = log_loss(y_test, pred_lr)
logloss_lr

0.6088414838550696

Random Forests

In [58]:
rf = RandomForestClassifier(n_estimators = 200,
                            min_samples_leaf = 10,
                            n_jobs = -1)

In [59]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [60]:
pred_rf = rf.predict_proba(X_test)[:,1]
logloss_rf = log_loss(y_test, pred_rf)

logloss_rf

0.5997685125147447

XGBoost

In [61]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [63]:
pred_xgb = xg.predict_proba(X_test)[:,1]
logloss_xg = log_loss(y_test, pred_xgb)


logloss_xg

0.6168490560157623

Neural Network

In [64]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [91]:
# define the keras model
model = Sequential()
model.add(Dense(900, input_dim=3966, activation='relu'))
model.add(Dense(240, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# 900 240 36 1

In [92]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [93]:

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=3, batch_size=50)

Epoch 1/3
3172/3172 [==============================] - 4s 1ms/step - loss: 0.6366 - accuracy: 0.6627
Epoch 2/3
3172/3172 [==============================] - 4s 1ms/step - loss: 0.5289 - accuracy: 0.7393
Epoch 3/3
3172/3172 [==============================] - 4s 1ms/step - loss: 0.3981 - accuracy: 0.8184


In [94]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

794/794 [==============================] - 0s 328us/step
Accuracy: 66.75
